# Physics and Simulation (PyBullet Examples)
## COMP 462/562 - Foundamentals of Robotic Manipulation

In [1]:
import pybullet as p
import pybullet_data as pd
import pybullet_utils.bullet_client as bc

### Connecting to the Physics Server

After importing the PyBullet module, the first thing to do is 'connecting' to the physics simulation. PyBullet is designed around a client-server driven API, with a client sending commands and a physics server returning the status.

There are two modes of connections: **DIRECT** and **GUI** </br>
The **DIRECT** connection sends the commands directly to the physics engine, without using any transport layer and no graphics visualization window.</br>
The **GUI** connection will create a new graphical user interface (GUI) with 3D OpenGL rendering.

If you want to use multiple independent simulations in parallel, you can use pybullet_utils.bullet_client. An instance of **BulletClient** has the same API as a pybullet instance.

In [2]:
bullet_client = bc.BulletClient(connection_mode=p.GUI)
bullet_client.setAdditionalSearchPath(pd.getDataPath())
bullet_client.setTimeStep(1./60.)

In [3]:
bullet_client.disconnect()

## Cube Example

In [4]:
bullet_client = bc.BulletClient(connection_mode=p.GUI)
bullet_client.setAdditionalSearchPath(pd.getDataPath())
bullet_client.setTimeStep(1./60.)

### Load / Create a (Multi) Body

The loadURDF will send a command to the physics server to load a physics model from a Universal Robot Description File (URDF).

In [5]:
planeID = bullet_client.loadURDF("plane.urdf")
print("The body ID for the plane:", planeID)

The body ID for the plane: 0


In [6]:
bullet_client.removeBody(planeID)

Although the recommended and easiest way to create stuff in the world is using the loading functions (loadURDF/SDF/MJCF/Bullet), you can also create collision and visual shapes programmatically and use them to create a body using createMultiBody.

In [7]:
colBoxID = bullet_client.createCollisionShape(p.GEOM_BOX, halfExtents=[0.1, 0.1, 0.1])
redvisBoxID = bullet_client.createVisualShape(p.GEOM_BOX, halfExtents=[0.1, 0.1, 0.1], rgbaColor=[1, 0, 0, 0.5])

redboxID = bullet_client.createMultiBody(baseMass=0.1, 
                                         baseCollisionShapeIndex=colBoxID,
                                         baseVisualShapeIndex=redvisBoxID,
                                         basePosition=[0, 0, 1], 
                                         baseOrientation=[0, 0, 0, 1]) # quaternion

print("The body ID for the red box:", redboxID)

The body ID for the red box: 0


### Real-Time Simulation

By default, the physics server will not step the simulation, unless you explicitly send a 'stepSimulation' command. </br>
This way you can maintain control determinism of the simulation.

In [8]:
while True:
    bullet_client.stepSimulation()

KeyboardInterrupt: 

Alternatively, it is also possible to run the simulation in real-time by letting the physics server automatically step the simulation according to its real-time-clock (RTC) using the setRealTimeSimulation command.

In [9]:
bullet_client.setRealTimeSimulation(True)

### Gravity

By default, there is no gravitational force enabled. setGravity lets you set the default gravity force for all objects.

In [10]:
bullet_client.setGravity(0, 0, -9.8)

You can create a static body by a zero baseMass.

In [11]:
greenvisBoxID = bullet_client.createVisualShape(p.GEOM_BOX, halfExtents=[0.1, 0.1, 0.1], rgbaColor=[0, 1, 0, 0.5])

greenboxID = bullet_client.createMultiBody(baseMass=0, 
                                           baseCollisionShapeIndex=colBoxID,
                                           baseVisualShapeIndex=greenvisBoxID,
                                           basePosition=[0, 0, 1], 
                                           baseOrientation=[0, 0, 0, 1])

print("The body ID for the green box:", greenboxID)

The body ID for the green box: 1


### Body Pose and Velocity

In [12]:
pos, orn = bullet_client.getBasePositionAndOrientation(redboxID)

print("Position of the red box:", pos)
print("Orientation of the red box:", orn)

Position of the red box: (0.0, 0.0, -55.48664697706147)
Orientation of the red box: (0.0, 0.0, 0.0, 1.0)


In [13]:
lin_vel, ang_vel = bullet_client.getBaseVelocity(redboxID)

print("Linear velocity of the red box:", lin_vel)
print("Angular velocity of the red box:", ang_vel)

Linear velocity of the red box: (0.0, 0.0, -15.156500549618151)
Angular velocity of the red box: (0.0, 0.0, 0.0)


In [14]:
bullet_client.resetBasePositionAndOrientation(redboxID, 
                                              posObj=[0, 0, 0.5], 
                                              ornObj=[0, 0, 0, 1])

### Save / Restore State

When you need deterministic simulation after restoring to a previously saved state, all important state information, including contact points, need to be stored.

In [15]:
stateID = bullet_client.saveState()

print("The state ID of the saved state:", stateID)
print("red box pose:", bullet_client.getBasePositionAndOrientation(redboxID))
print("red box velocity:", bullet_client.getBaseVelocity(redboxID))

The state ID of the saved state: 0
red box pose: ((0.0, 0.0, -19.72181958221615), (0.0, 0.0, 0.0, 1.0))
red box velocity: ((0.0, 0.0, -13.682200618722653), (0.0, 0.0, 0.0))


In [16]:
bullet_client.restoreState(stateID)

print("red box pose:", bullet_client.getBasePositionAndOrientation(redboxID))
print("red box velocity:", bullet_client.getBaseVelocity(redboxID))

red box pose: ((0.0, 0.0, -19.72181958221615), (0.0, 0.0, 0.0, 1.0))
red box velocity: ((0.0, 0.0, -13.682200618722653), (0.0, 0.0, 0.0))


### Control

We wil create a blue box which has one joint. The joint connects its virtual base (invisible) and one link (i.e., the blue box).

In [17]:
bluevisBoxID = bullet_client.createVisualShape(p.GEOM_BOX, halfExtents=[0.1, 0.1, 0.1], rgbaColor=[0, 0, 1, 0.5])

blueboxID = bullet_client.createMultiBody(baseMass=0,
                                          baseCollisionShapeIndex=-1,
                                          baseVisualShapeIndex=-1,
                                          basePosition=[0, 0, 0], 
                                          baseOrientation=[0, 0, 0, 1],
                                    linkMasses=[0.2],
                                    linkCollisionShapeIndices=[colBoxID],
                                    linkVisualShapeIndices=[bluevisBoxID],
                                    linkPositions=[[0, 0, 0.5]],
                                    linkOrientations=[[0, 0, 0, 1]],
                                    linkInertialFramePositions=[[0, 0, 0]],
                                    linkInertialFrameOrientations=[[0, 0, 0, 1]],
                                    linkParentIndices=[0],
                                    #linkJointTypes=[p.JOINT_PRISMATIC],
                                    linkJointTypes=[p.JOINT_REVOLUTE],
                                    linkJointAxis=[[1, 0, 0]]
                                    )

In [18]:
bullet_client.removeBody(blueboxID)

In [19]:
print("Number of joints of the blue box:", 
      bullet_client.getNumJoints(blueboxID))

print("Number of joints of the red box:", 
      bullet_client.getNumJoints(redboxID))

Number of joints of the blue box: 0
Number of joints of the red box: 0


In [24]:
jnt_info = bullet_client.getJointInfo(bodyUniqueId=blueboxID,
                                      jointIndex=0)
print(jnt_info)

error: GetJointInfo failed.

We can control a robot by setting a desired control mode for one or more joint motors. During the stepSimulation the physics engine will simulate the motors to reach the given target value that can be reached within the maximum motor forces

In [21]:
bullet_client.setJointMotorControl2(bodyUniqueId=blueboxID,
                                    jointIndex=0,
                                    controlMode=p.POSITION_CONTROL,
                                    targetPosition=1.5)

In [22]:
bullet_client.setJointMotorControl2(bodyUniqueId=blueboxID,
                                    jointIndex=0,
                                    controlMode=p.VELOCITY_CONTROL,
                                    targetVelocity=-1.5)

In [23]:
jnt_state = bullet_client.getJointState(bodyUniqueId=blueboxID, 
                                        jointIndex=0)

print("jointPosition:", jnt_state[0])
print("jointVelocity:", jnt_state[1])
print("jointReactionForces:", jnt_state[2])
print("appliedJointMotorTorque:", jnt_state[3])

jointPosition: -3.3250000070506225
jointVelocity: -1.5
jointReactionForces: (0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
appliedJointMotorTorque: -0.00019999999552965168


You can apply a force or torque to a body using **applyExternalForce** and **applyExternalTorque**. Note that this method will only work when explicitly stepping the simulation using stepSimulation. After each simulation step, the external forces are cleared to zero.

In [24]:
bullet_client.setRealTimeSimulation(False)

You can change the properties such as mass, friction and restitution coefficients using changeDynamics.

In [25]:
bullet_client.changeDynamics(bodyUniqueId=redboxID, 
                             linkIndex=-1,
                             lateralFriction=0.1)
                             #lateralFriction=0.5)

In [30]:
bullet_client.resetBasePositionAndOrientation(redboxID, [0, 0, 0.1], [0, 0, 0, 1])

In [27]:
import time

while True:
    bullet_client.applyExternalForce(objectUniqueId=redboxID, 
                                     linkIndex=-1, 
                                     forceObj=[0.5, 0, 0],
                                     posObj=[0, 0, 0],
                                     #flags=p.LINK_FRAME
                                     flags=p.WORLD_FRAME
                                    )
    bullet_client.stepSimulation()
    time.sleep(0.01)

KeyboardInterrupt: 

In [29]:
while True:
    bullet_client.applyExternalTorque(objectUniqueId=redboxID, 
                                     linkIndex=-1, 
                                     torqueObj=[0, 0, 0.1],
                                     #flags=p.LINK_FRAME
                                     flags=p.WORLD_FRAME
                                    )
    bullet_client.stepSimulation()
    time.sleep(0.01)

KeyboardInterrupt: 

### Collision Detection

In [31]:
bullet_client.setRealTimeSimulation(True)

In [32]:
bullet_client.resetBasePositionAndOrientation(greenboxID, [0, 0, 0.1], [0, 0, 0, 1])

In [33]:
bullet_client.removeBody(blueboxID)

In [35]:
conpts = bullet_client.getContactPoints(greenboxID, redboxID)
print("Number of contact points:", len(conpts))

Number of contact points: 4


In [36]:
first_conpt = conpts[0]
print(first_conpt)

(0, 2, 1, -1, -1, (0.05403944174479369, 0.1, 0.0), (0.05403944174479367, 0.09996994526394092, 0.0), (-0.0, -1.0, -0.0), -3.005473605909026e-05, 0.002874484958493922, 1.717898333654484e-08, (-1.0, 0.0, 0.0), 0.00014372424689801665, (-0.0, -0.0, 1.0))


In [37]:
print("position on the green box:", first_conpt[5])
print("position on the red box:", first_conpt[6])
print("contact distance:", first_conpt[8])
print("normal force:", first_conpt[9])

position on the green box: (0.05403944174479369, 0.1, 0.0)
position on the red box: (0.05403944174479367, 0.09996994526394092, 0.0)
contact distance: -3.005473605909026e-05
normal force: 0.002874484958493922


## Panda Example

In [38]:
bullet_client.removeBody(greenboxID)
bullet_client.removeBody(redboxID)

In [39]:
pandaID = bullet_client.loadURDF("franka_panda/panda.urdf", 
                                 [0, 0, 0], 
                                 [0, 0, 0, 1], 
                                 useFixedBase=True)

print("The body ID of the Panda robot:", pandaID)

The body ID of the Panda robot: 1


### Get joint info / state

In [40]:
print("Number of joints of Panda:", bullet_client.getNumJoints(pandaID))

Number of joints of Panda: 12


In [47]:
jnt_info = bullet_client.getJointInfo(bodyUniqueId=pandaID,
                                      jointIndex=10)

print("jointName:", jnt_info[1])
print("jointType:", jnt_info[2])
print("linkName:", jnt_info[12])
print("jointLowerLimit:", jnt_info[8])
print("jointUpperLimit:", jnt_info[9])

jointName: b'panda_finger_joint2'
jointType: 1
linkName: b'panda_rightfinger'
jointLowerLimit: 0.0
jointUpperLimit: 0.04


In [49]:
jnt_state = bullet_client.getJointState(bodyUniqueId=pandaID, 
                                        jointIndex=4)

print("jointPosition:", jnt_state[0])
print("jointVelocity:", jnt_state[1])
print("jointReactionForces:", jnt_state[2])
print("appliedJointMotorTorque:", jnt_state[3])

jointPosition: -0.8448237532674937
jointVelocity: -9.371899239823955e-05
jointReactionForces: (0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
appliedJointMotorTorque: 0.024729912831059804


### Set joint angle (and velocity)

In [50]:
target_angles = [0.0, -0.7853981633974483, 0.0, -2.356194490192345,
                 0.0, 1.5707963267948966, 0.7853981633974483]

for i in range(7):
    bullet_client.resetJointState(pandaID, i, target_angles[i])

### End-effector state and Jacobian

In [52]:
ee_state = bullet_client.getLinkState(pandaID, linkIndex=11)

print("Position of the end-effector:", ee_state[4])
print("Orientation of the end-effector:", ee_state[5])

Position of the end-effector: (0.276626855134964, -0.011761602014303207, 0.46511733531951904)
Orientation of the end-effector: (0.9952887892723083, -0.011828192509710789, -0.09250179678201675, 0.02652869187295437)


In [59]:
import numpy as np

jstates = bullet_client.getJointStates(pandaID, 
                                       range(bullet_client.getNumJoints(pandaID)))
jinfos = [bullet_client.getJointInfo(pandaID, i) \
          for i in range(bullet_client.getNumJoints(pandaID))]
jstates = [j for j, i in zip(jstates, jinfos) if i[3] > -1]
mjpos = [state[0] for state in jstates]
Jt, Jr = bullet_client.calculateJacobian(pandaID, 11, 
                                         localPosition=[0.0, 0.0, 0.0], 
                                         objPositions=mjpos,
                                         objVelocities=[0.0]*len(mjpos),
                                         objAccelerations=[0.0]*len(mjpos))
Jt, Jr = np.array(Jt)[:, 0:7], np.array(Jr)[:, 0:7]
J = np.vstack((Jt, Jr))

print("Size of J:", J.shape)
print("Jacobian matrix:", J)

Size of J: (6, 7)
Jacobian matrix: [[ 2.94568637e-02  1.17147083e-01  2.33699613e-02  1.74915523e-01
   1.65058918e-03  2.29433703e-01  0.00000000e+00]
 [ 2.36314239e-01 -1.83136649e-03  2.54969441e-01 -2.82957705e-03
   2.27836270e-01 -2.39376118e-03 -0.00000000e+00]
 [-6.93889390e-18 -2.36745812e-01  1.69675808e-02  3.82824249e-01
  -2.55845938e-02 -6.51504704e-03  0.00000000e+00]
 [-1.11022302e-16  1.56311425e-02 -6.58614160e-01 -1.58175642e-02
   9.96981711e-01 -7.19919261e-03 -4.10327258e-01]
 [-1.38777878e-17  9.99877826e-01  1.02961497e-02 -9.99874881e-01
  -1.57593858e-02 -9.93728311e-01 -9.88291955e-02]
 [ 1.00000000e+00  4.89661089e-12  7.52410379e-01 -1.63222448e-04
  -7.60204547e-02  1.11589499e-01 -9.06567334e-01]]


### Joint-space control

In [63]:
bullet_client.setJointMotorControlArray(bodyUniqueId=pandaID,
                                        jointIndices=range(7),
                                        controlMode=p.VELOCITY_CONTROL,
                                        targetVelocities=[0.1]*7)

In [64]:
bullet_client.setJointMotorControlArray(bodyUniqueId=pandaID,
                                        jointIndices=range(7),
                                        controlMode=p.POSITION_CONTROL,
                                        targetVelocities=[0]*7)

In [65]:
bullet_client.disconnect()